In [ ]:
from pathlib import Path
import json
import numpy as np
import matplotlib.pyplot as plt

RUN_DIR = Path(r"C:\Users\gerry\OneDrive\ESSEC-CS\M2\Foundations_Deep_Learning\Project\yolo_run_20260122_2200")
RESNET_DIR = RUN_DIR / "resnet18"
METRICS_JSON = RESNET_DIR / "resnet18_metrics.json"
META_JSON    = RESNET_DIR / "resnet18_meta.json"
OUT_DIR      = RESNET_DIR / "analysis"
OUT_DIR.mkdir(parents=True, exist_ok=True)

def safe_div(a, b):
    return a / b if b != 0 else 0.0

def compute_metrics_from_cm(cm: np.ndarray, class_names):
    # cm rows=true, cols=pred
    supports = cm.sum(axis=1)
    pred_totals = cm.sum(axis=0)
    total = cm.sum()

    acc = safe_div(np.trace(cm), total)
    recalls = np.array([safe_div(cm[i, i], supports[i]) for i in range(len(class_names))])
    precisions = np.array([safe_div(cm[i, i], pred_totals[i]) for i in range(len(class_names))])
    f1s = np.array([
        safe_div(2 * precisions[i] * recalls[i], precisions[i] + recalls[i])
        for i in range(len(class_names))
    ])

    bal_acc = float(recalls.mean())
    macro_f1 = float(f1s.mean())

    return {
        "accuracy": float(acc),
        "balanced_accuracy": float(bal_acc),
        "macro_f1": macro_f1,
        "per_class": {
            class_names[i]: {
                "precision": float(precisions[i]),
                "recall": float(recalls[i]),
                "f1": float(f1s[i]),
                "support": int(supports[i]),
            }
            for i in range(len(class_names))
        },
        "predicted_distribution": {
            class_names[i]: int(pred_totals[i]) for i in range(len(class_names))
        },
        "total": int(total),
    }

def plot_confusion_matrix(cm, class_names, out_path, normalize=False, title=None):
    cm_plot = cm.astype(float)
    if normalize:
        row_sums = cm_plot.sum(axis=1, keepdims=True)
        cm_plot = np.divide(cm_plot, row_sums, out=np.zeros_like(cm_plot), where=row_sums != 0)

    fig, ax = plt.subplots(figsize=(5.5, 4.5))
    im = ax.imshow(cm_plot)

    ax.set_xticks(range(len(class_names)))
    ax.set_yticks(range(len(class_names)))
    ax.set_xticklabels(class_names, rotation=45, ha="right")
    ax.set_yticklabels(class_names)

    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    ax.set_title(title or ("Confusion Matrix" + (" (Normalized)" if normalize else "")))

    # annotate cells
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            if normalize:
                txt = f"{cm_plot[i, j]:.2f}"
            else:
                txt = str(int(cm[i, j]))
            ax.text(j, i, txt, ha="center", va="center")

    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    fig.tight_layout()
    fig.savefig(out_path, dpi=200)
    plt.close(fig)

def main():
    metrics = json.loads(METRICS_JSON.read_text(encoding="utf-8"))
    meta = json.loads(META_JSON.read_text(encoding="utf-8"))

    class_names = metrics.get("classes_in_folder_order", ["left", "right"])
    cm = np.array(metrics["confusion_matrix_rows_true_cols_pred"], dtype=int)

    summary = compute_metrics_from_cm(cm, class_names)

    # Save text report
    report_lines = []
    report_lines.append(f"Best val acc: {metrics['best_val_acc']:.6f}")
    report_lines.append(f"Best phase: {metrics['best_phase']}, epoch in phase: {metrics['best_epoch_in_phase']}")
    report_lines.append(f"Accuracy: {summary['accuracy']:.6f}")
    report_lines.append(f"Balanced accuracy: {summary['balanced_accuracy']:.6f}")
    report_lines.append(f"Macro F1: {summary['macro_f1']:.6f}")
    report_lines.append("")
    report_lines.append("Per-class metrics:")
    for cls in class_names:
        m = summary["per_class"][cls]
        report_lines.append(
            f"  {cls:>5s} | P={m['precision']:.3f}  R={m['recall']:.3f}  F1={m['f1']:.3f}  support={m['support']}"
        )
    report_lines.append("")
    report_lines.append("Predicted distribution:")
    for cls in class_names:
        report_lines.append(f"  pred_{cls}: {summary['predicted_distribution'][cls]}")

    (OUT_DIR / "resnet_classification_report.txt").write_text("\n".join(report_lines), encoding="utf-8")

    # Save plots
    plot_confusion_matrix(
        cm, class_names,
        OUT_DIR / "resnet_confusion_matrix.png",
        normalize=False,
        title="ResNet18 Team Confusion Matrix (Counts)"
    )
    plot_confusion_matrix(
        cm, class_names,
        OUT_DIR / "resnet_confusion_matrix_normalized.png",
        normalize=True,
        title="ResNet18 Team Confusion Matrix (Row-normalized)"
    )

    # Also save a JSON summary for easy use in LaTeX/table generation
    (OUT_DIR / "resnet_metrics_derived.json").write_text(json.dumps(summary, indent=2), encoding="utf-8")

    print("Saved:")
    print(" -", OUT_DIR / "resnet_classification_report.txt")
    print(" -", OUT_DIR / "resnet_confusion_matrix.png")
    print(" -", OUT_DIR / "resnet_confusion_matrix_normalized.png")
    print(" -", OUT_DIR / "resnet_metrics_derived.json")

if __name__ == "__main__":
    main()


Saved:
 - C:\Users\gerry\OneDrive\ESSEC-CS\M2\Foundations_Deep_Learning\Project\yolo_run_20260122_2200\resnet18\analysis\resnet_classification_report.txt
 - C:\Users\gerry\OneDrive\ESSEC-CS\M2\Foundations_Deep_Learning\Project\yolo_run_20260122_2200\resnet18\analysis\resnet_confusion_matrix.png
 - C:\Users\gerry\OneDrive\ESSEC-CS\M2\Foundations_Deep_Learning\Project\yolo_run_20260122_2200\resnet18\analysis\resnet_confusion_matrix_normalized.png
 - C:\Users\gerry\OneDrive\ESSEC-CS\M2\Foundations_Deep_Learning\Project\yolo_run_20260122_2200\resnet18\analysis\resnet_metrics_derived.json
